# Generator 

In [15]:
def count_down(x):
    print('Why, am I ignore ?\n')
    for i in range(x, 0, -1):
        yield i

In [34]:
gener_1 = count_down(10)
gener_1

<generator object count_down at 0x7f5bdc0d2ed0>

In [17]:
for i in gener_1:
    print(i)

Why, am I ignore ?

10
9
8
7
6
5
4
3
2
1


In [35]:
next(gener_1)

Why, am I ignore ?



10

- yield produces a value, but suspends the function
- Function resumes on next call to next()

In [36]:
next(gener_1)

9

In [37]:
next(gener_1)

8

## Coroutines

In [1]:
def grep(pattern):
    print("Looking for %s" % pattern)
    while True:
        line = yield
        if pattern in line:
            print(line),
        else:
            print(f"There is no \"{pattern}\" word in your input \"{line}\" ")

In [2]:
g = grep("python")

In [3]:
next(g)

Looking for python


In [4]:
g.send("Yeah, but no, but yeah, but no")

There is no "python" word in your input "Yeah, but no, but yeah, but no" 


In [5]:
g.send("A series of tubes")

There is no "python" word in your input "A series of tubes" 


In [6]:
g.send("python generators rock!")

python generators rock!


All coroutines must be "primed" by first calling `next()` (or `send(None)`)m if not one will get tis kind of error :
<img src="./images/error_0.png">

### Decorator
We can use decorators to make sure to avoid this kind of errors

In [26]:
def coroutine(func):
    def start(*args,**kwargs):
        cr = func(*args,**kwargs)
        next(cr)
        return cr
    return start

In [27]:
@coroutine
def grep(pattern):
    print("Looking for %s" % pattern)
    while True:
        line = yield
        if pattern in line:
            print(line),
        else:
            print(f"There is no \"{pattern}\" word in your input \"{line}\" ")

In [28]:
g = grep("python")
g
g.send("Yeah, but no, but yeah, but no")

Looking for python
There is no "python" word in your input "Yeah, but no, but yeah, but no" 


Alwars remember to close your coroutines by typing `.close()`or it'll runn indefinitly

In [30]:
g.close()

You can see that there is no message to let us know that the coroutines is close, it a good practice to use a `try: + except` to catch the `.close()`.

In [31]:
@coroutine
def grep(pattern):
    print("Looking for %s" % pattern)
    try:
        while True:
            line = yield
            if pattern in line:
                print(line),
            else:
                print(f"There is no \"{pattern}\" word in your input \"{line}\" ")
    except GeneratorExit:
        print("Going away. Goodbye")

In [32]:
g = grep("python")
g
g.send("Yeah, but no, but yeah, but no")

Looking for python
There is no "python" word in your input "Yeah, but no, but yeah, but no" 


In [33]:
g.close()

Going away. Goodbye


___
Note:
- Despite some similarities, Generators and coroutines are basically two different concepts
- Generators produce values
- Coroutines tend to consume values
- It is easy to get sidetracked because methods meant for coroutines are sometimes described as a way to tweak generators that are in the process of producing an iteration pattern (i.e., resetting its value). This is mostly bogus.

### A "generator" that produces and receives values

In [38]:
def countdown(n):
    print("Counting down from", n)
    while n >= 0:
        newvalue = (yield n)
        # If a new value got sent in, reset n with it
        if newvalue is not None:
            n = newvalue
        else:
            n -= 1
            
c = countdown(5)
for n in c:
    print(n)
    if n == 5:
        c.send(2)

Counting down from 5
5
1
0


## yield from
`yield from` iterable is essentially just a shortened form of `for item in iterable: yield item:`

In [39]:
def chain(*iterables):
    for it in iterables:
        for value in it:
            yield value

In [40]:
list(chain("hello", ["world"], ("tuple", " of ", "values.")))

['h', 'e', 'l', 'l', 'o', 'world', 'tuple', ' of ', 'values.']

In [55]:
def chain(*iterables):
    for it in iterables:
        yield from it

In [56]:
list(chain("hello", ["world"], ("tuple", " of ", "values.")))

['h', 'e', 'l', 'l', 'o', 'world', 'tuple', ' of ', 'values.']

In [58]:
def g(x):
    yield from range(x, 0, -1)
    yield from range(x)
    
list(g(5))

[5, 4, 3, 2, 1, 0, 1, 2, 3, 4]

In [59]:
def all_powers(n, pow):
    yield from (n ** i for i in range(pow + 1))

In [61]:
list(all_powers(2, 3))

[1, 2, 4, 8]

In [64]:
def all_powers(n, pow):
    return [n ** i for i in range(pow + 1)]

In [65]:
all_powers(2, 3)

[1, 2, 4, 8]

## Capturing the value returned by a sub-generator

In [143]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('generators_yieldfrom_2') 

def sequence(name, start, end):
    logger.info("%s started at %i", name, start)
    yield from range(start, end)
    logger.info("%s finished at %i", name, end)
    return end

def main():
    step1 = yield from sequence("first", 0, 5)
    step2 = yield from sequence("second", step1, 10)
    return step1 + step2

In [144]:
list(sequence("first", 0, 5))

INFO:generators_yieldfrom_2:first started at 0
INFO:generators_yieldfrom_2:first finished at 5


[0, 1, 2, 3, 4]

In [130]:
g = main()
next(g)

INFO:generators_yieldfrom_2:first started at 0


0

In [131]:
next(g)

1

In [132]:
next(g)

2

In [133]:
next(g)

3

In [134]:
next(g)

4

In [135]:
next(g)

INFO:generators_yieldfrom_2:first finished at 5
INFO:generators_yieldfrom_2:second started at 5


5

In [136]:
next(g)

6

In [137]:
next(g)

7

In [138]:
next(g)

8

In [139]:
next(g)

9

In [140]:
next(g)

INFO:generators_yieldfrom_2:second finished at 10


StopIteration: 15

<h1>About the Authors:</h1> 

<a href="https://skabongo.github.io/">Salomon Kabongo</a>, Master degree student at <a href="https://aims.ac.za/">the African Master in Machine Intelligence (AMMI, Ghana)</a> his research focused on the use machine learning technique in the field of Natural Language Processing, learn more about him [here](https://skabongo.github.io/) or https://skabongo.github.io/.

**References :** ... 

Copyright &copy; 2020. This notebook and its source code are released under the terms of the <a href="https://www.apache.org/licenses/LICENSE-2.0">Apache License 2.0</a>.